In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
from IPython.display import HTML
from tools import python_to_html

In [5]:
from GomokuTools2 import GomokuTools, NH9x9, Heuristics

In [6]:
A,B,C,D,E,F,G,H,I,K,L,M,N,O,P,Q,R,S,T = \
    1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
BLACK=0
WHITE=1

## The 9x9 Neighbourhood
The 9x9 neighbourhood accounts for all the positions on the board that have a direct influence on the value of the central position, i.e. stones on those positions could participate in a common line with the central position.

In [7]:
nh = NH9x9()
nh

|                 |
|                 |
|                 |
|                 |
|        *        |
|                 |
|                 |
|                 |
|                 |

Let's put a black stone at a distance of 2 in to the north-east direction and a white stone farthest into the south-west.

In [8]:
nh.register(BLACK, 'ne', distance=2).register(WHITE, 'sw', distance=4)

|                 |
|                 |
|            x    |
|                 |
|        *        |
|                 |
|                 |
|                 |
|o                |

Display as bit array

In [9]:
nh.as_bits()

[[[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0]],
 [[0, 0, 0, 0, 0, 1, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0]],
 [[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0]],
 [[0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0]]]

Register complete lines

In [10]:
nh=NH9x9().setline_xo('n', 'xo.x.x..').setline_xo('e', 'o.x.x...').setline_xo('ne', '.......o')
nh

|                o|
|                 |
|        x        |
|                 |
|o   x   * x      |
|        x        |
|                 |
|        o        |
|        x        |

In [11]:
nh.unregister(WHITE, 'ne', distance=4)

|                 |
|                 |
|        x        |
|                 |
|o   x   * x      |
|        x        |
|                 |
|        o        |
|        x        |

---
## Heuristics: Learning from Humans
Alpha zero started from scratch and explored the world of go with no previous knowledge but the rules. Typically though, some human knowledge can jump-start the learning process. That's why we give our agent a head-start with some truly not-so-rigorous (but still quite smart) heuristics. We'll make sure though that the agent will be able to extend its understanding beyond its initial knowlegde and eventually even abandon the heuristics in favour of what it has learned by itself.

The heuristic score tries to naively measure the *value* of a position, i.e. the importance with which one should consider putting a stone onto it.

```Heuristics``` uses the 2-byte = 2x 8 bit representation of a line of 9 positions. For example:
```
[[0, 0, 1, 0,      0, 0, 0, 0],
 [0, 0, 0, 0,      0, 0, 1, 1]]
```
means there is a black stone in one direction (say: left) with distance 2 and there are 2 white stones in the opposite direction (say: right) with distances 3 and 4. The line could as well be represented by a string like ```..x.*..oo ```, which would be understood by method ```setline_xo``` of class ```NH9x9```

The actual direction of the viewpoint don't matter here. What matters is that the position marked by ```*``` would create an wide-split-3 threat in favor of white.

In [22]:
h=Heuristics()

In [23]:
n=NH9x9().setline_xo(2, 'ox.o.o..').setline_xo(0, 'x.o.o...')
classification = h.classify_nh(n, score_for=0)
n, classification, h.describe(classification), h.soft_values(n)

(|                 |
 |                 |
 |        o        |
 |                 |
 |x   o   * o      |
 |        o        |
 |                 |
 |        x        |
 |        o        |,
 [(0, 0, 6), (4, 4, 5)],
 ['Black: defendable', 'White: 2 double open 2s - move or lose in 3'],
 [8, 5184])

In [24]:
n=NH9x9().setline_xo(2, 'o...xxx.')
classification = h.classify_nh(n, score_for=0)
n, classification, h.describe(classification), h.soft_values(n)

(|                 |
 |        x        |
 |        x        |
 |        x        |
 |        *        |
 |                 |
 |                 |
 |                 |
 |        o        |,
 [(7, 0, 3), (1, 0, 6)],
 ['Black: double open 3 - move or lose in 2', 'White: defendable'],
 [14088, 256])

---
### Counting stones

```Heuristics``` establishes its functions upon lower-level building blocks such as the ```cscore``` *count score* function:

In [25]:
h.cscore?

Signature: h.cscore(line, c=0, edges=(None, None), cap=2)
Docstring:
count how many sub-lines of 5 come with the max number of stones
Example: "oo.x*xx.." : The max num of blacks if obviously 3. And there are
         two different adversary-free sub-lines counting three, namely '.x*xx' and 'x*xx.'.
         Thus the cscore would be (3,2)

Args:
    line: 8x2 integer array that represents the stones 
    c:  color: 0 to look at black, 1 to consider white
File:      ~/workspace/tutorials/other_stuff/DeepGomoku/GomokuTools2.py
Type:      method


In [34]:
n=NH9x9().setline_xo('nw', 'ox.o.o.x').setline_xo('e', 'x.o.oo..')
n

|x                |
|                 |
|    o            |
|                 |
|x   o   * o o    |
|          o      |
|                 |
|              x  |
|                o|

In [35]:
h.cscore(n.get_line('e'), c=WHITE)

(3, 1)

Here, the dominant feature of the eastern line is made of 3 whites and only 1 way of completing them to a row of 5.

---
### Adversary-free ranges

In [38]:
h.f_range?

Signature: h.f_range(line, c=0, edges=(None, None))
Docstring:
The largest adversary-free range within a given line

Args:
    line: 8x2 integer array that represents the stones
    c:    0 to look at black, 1 to consider white
File:      ~/workspace/tutorials/other_stuff/DeepGomoku/GomokuTools2.py
Type:      method


In [37]:
h.f_range(n.get_line('e'), c=WHITE)

array([0, 1, 0, 1, 1, 0, 0])